In [ ]:
import cartopy.crs as ccrs
import numpy as np
import pandas as pd
import xarray as xr
import holoviews as hv
import geoviews as gv
import datashader as dsh

hv.extension('bokeh')

%opts QuadMesh Image [width=600 height=400 colorbar=True] Feature [apply_ranges=False]

In geographical applications grids and meshes of different kinds are very common and for visualization and analysis it is often very important to be able to regrid them in different ways. Regridding can refer both to upsampling and downsampling a grid or mesh, which is achieved through interpolation and aggregation respectively.

Naive approaches to regridding treat the space as flat, which is often simpler but can also give incorrect results when working with a flat earth. In this user guide we will summarize how to work with different grid types including rectilinear, curvilinear grids and trimeshes. Additionally we will discuss different approaches to regridding working based on the assumption of a flat earth (using [datashader](http://datashader.org/)) and a spherical earth ([xESMF](http://xesmf.readthedocs.io/en/latest/Curvilinear_grid.html)).

# Rectilinear grids

Rectilinear grids are one of the most standard formats and are defined by regularly sampled coordinates along the two axes. The ``air_temperature`` dataset provided by xarray and used throughout the GeoViews documentation provides a good example.

In [ ]:
ds = xr.tutorial.load_dataset('air_temperature').isel(time=slice(0, 5))
# Ensure data is in range (-180, 180)
ds['lon'] = np.where(ds.lon>180, ds.lon-360, ds.lon)
ds

As we already know from the [Gridded Dataset](./user_guide/Gridded_Datasets_II.ipynb) sections, an xarray of this kind can easily be wrapped in a GeoViews dataset:

In [ ]:
gvds = gv.Dataset(ds)
gvds

We can also easily plot this data by using the ``.to`` method to create set of ``Image`` elements:

In [ ]:
%opts Image (cmap='viridis') 
images = gvds.to(gv.Image, ['lon', 'lat'], dynamic=True)
images * gv.feature.coastline

### Datashader

In [ ]:
from holoviews.operation.datashader import regrid
regrid(images, interpolation='linear') * gv.feature.coastline

### xESMF

In [ ]:
from geoviews.operation.regrid import weighted_regrid
weighted_regrid(images, interpolation='nearest_s2d').redim.range(air=(230, 300)) * gv.feature.coastline

# Curvilinear Grids

Curvilinear grids are another very common mesh type, which are usually defined by multi-dimensional coordinate arrays:

In [ ]:
ds = xr.tutorial.load_dataset('rasm')
# Adjust the longitudes so they are in the range -180 to 180
ds.xc.values[ds.xc.values>180] -= 360 
ds

Just like the rectilinear grids GeoViews understands this kind of data natively. So we again wrap this dataset in a ``gv.Dataset`` and define a fixed range for the ``Tair`` values:

In [ ]:
gvds = gv.Dataset(ds).redim.range(Tair=(-25, 25))
gvds

Now we can plot this data directly as a ``gv.QuadMesh``, however this is generally quite slow, especially when we are working with bokeh where each grid point is rendered as a distinct polygon. We will therefore downsample the data by 4x along both dimensions:

In [ ]:
%opts Image QuadMesh (cmap='RdBu_r')
quadmeshes = gvds.to(gv.QuadMesh, ['xc', 'yc'], dynamic=True)
quadmeshes.map(lambda x: x.clone(x.data.Tair[::4, ::4]), gv.QuadMesh) * gv.feature.coastline

If we want to explore the whole dataset we therefore don't have much of a choice except for regridding it onto a rectilinear grid, which can be rendered much more efficiently. Once again we have the option of using the datashader based approach or the more correct xESMF based approach.

### Datashader

To regrid a ``QuadMesh`` using GeoViews we can import the ``rasterize`` operation. In the background the operation will convert the ``QuadMesh`` into a ``TriMesh``, which datashader understands. To optimize this conversion so it occurs only when aggregating an Image for the first time we can activate the ``precompute`` option. Additionally we have to define an aggregator, in this case to compute the mean ``Tair`` value in a pixel:

In [ ]:
from holoviews.operation.datashader import rasterize
rasterize(quadmeshes, precompute=True, aggregator=dsh.mean('Tair')) * gv.feature.coastline

### xESMF

The xESMF based approach stores generates a sparse weight matrix which weights each grid cell appropriately. Optionally this weight matrix can be cached to disk to speed up subsequent aggregation, to enable this optimization enable ``reuse_weights`` (and optionally define a custom filepattern).

In [ ]:
from geoviews.operation.regrid import weighted_regrid
weighted_regrid(quadmeshes, reuse_weights=True) * gv.feature.coastline

# TriMesh

Another commonly used mesh/grid type are trimeshes, here we will load a 3dm file and demonstrate how to visualize it using HoloViews:

In [ ]:
fpath = './Example_Meshes/Chesapeake_and_Delaware_Bays.3dm'
all_df = pd.read_table(fpath, delim_whitespace=True, header=None, skiprows=1,
                       names=('row_type', 'cmp1', 'cmp2', 'cmp3', 'val'), index_col=1)
all_df.head()

Before we can work with this data we have to process it a bit, splitting the data into vertices and triangles:

In [ ]:
conns = all_df[all_df['row_type'].str.lower() == 'e3t'][['cmp1', 'cmp2', 'cmp3']].values.astype(int) - 1
pts = all_df[all_df['row_type'].str.lower() == 'nd'][['cmp1', 'cmp2', 'cmp3']].values.astype(float)
pts[:, 2] *= -1
verts = pd.DataFrame(pts, columns=['x', 'y', 'z'])
tris = pd.DataFrame(conns, columns=['v0', 'v1', 'v2'])
print("Triangles: %d\nVertices: %d" % (len(tris), len(verts)))

### Datashader

This is a reasonably large mesh with >1 million triangles and cannot easily be displayed directly with matplotlib or especially bokeh. Therefore we will once again regrid this dataset onto a rectilinear grid. Additionally, to speed up plotting of meshes, we can do two things, first of all since all data displayed with bokeh is first projected to Web Mercator coordinates we can project the mesh immediately, secondly we can use the ``precompute`` during rasterization, which will cache an optimized mesh datastructure:

In [ ]:
%%opts Image [width=800 height=500] (cmap='viridis')
trimesh = gv.operation.project_graph(gv.TriMesh((tris, verts)))
tiles = gv.WMTS('https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png')
tiles * rasterize(trimesh, precompute=True, aggregator=dsh.mean('z'))